In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, LinearRing

# ----- Step 1: Load your GeoJSON -----
gdf = gpd.read_file("your_file.geojson")
geom_series = gdf.geometry

# ----- Step 2: Chaikin smoothing function -----
def chaikin_smooth(coords, iterations=2):
    """
    coords: list of (x, y) tuples
    iterations: number of smoothing passes
    Returns new list of smoothed coordinates
    """
    for _ in range(iterations):
        new_coords = []
        for i in range(len(coords)-1):
            p0, p1 = coords[i], coords[i+1]
            Q = (0.75*p0[0] + 0.25*p1[0], 0.75*p0[1] + 0.25*p1[1])
            R = (0.25*p0[0] + 0.75*p1[0], 0.25*p0[1] + 0.75*p1[1])
            new_coords += [Q, R]
        # Close the ring
        new_coords.append(new_coords[0])
        coords = new_coords
    return coords

# ----- Step 3: Apply smoothing to each geometry -----
def smooth_polygon(geom, iterations=2, simplify_tol=None):
    if geom.geom_type == 'Polygon':
        # Simplify first (optional)
        if simplify_tol:
            geom = geom.simplify(simplify_tol, preserve_topology=True)
        # Apply Chaikin smoothing to exterior
        new_exterior = LinearRing(chaikin_smooth(list(geom.exterior.coords), iterations))
        # Smooth interiors (holes) if any
        new_interiors = [LinearRing(chaikin_smooth(list(ring.coords), iterations)) for ring in geom.interiors]
        return Polygon(new_exterior, new_interiors)
    elif geom.geom_type == 'MultiPolygon':
        return MultiPolygon([smooth_polygon(p, iterations, simplify_tol) for p in geom.geoms])
    else:
        return geom

# Example: 2 Chaikin iterations, optional 3-meter simplification
smoothed_series = geom_series.apply(lambda g: smooth_polygon(g, iterations=2, simplify_tol=3))

# ----- Step 4: Save as GeoJSON -----
gdf_smoothed = gpd.GeoDataFrame(geometry=smoothed_series, crs=gdf.crs)
gdf_smoothed.to_file("smoothed_output.geojson", driver="GeoJSON")
